# Taller 1: Regresión con Random Forest y Gradient Boosting


En este taller se aplicará ensambes de árboles de decisión tipo random forest y gradient boosting a un conjunto de datos de jugadores del video juego FIFA 19. Los datos utilizados son un subconjunto modificado de los dataset de Kaggle de FIFA 19 y se encuentran en el archivo `players.csv`. La tarea a realizar consiste en predecir el salario (wage) de los jugadores.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.decomposition import PCA
from sklearn.compose import ColumnTransformer
from matplotlib import pyplot as plt
from sklearn.preprocessing import (
    MinMaxScaler,
    OneHotEncoder,
    OrdinalEncoder,
    StandardScaler,
    PolynomialFeatures,
    FunctionTransformer,
)

In [2]:
random_state=42

## Preprocesamiento de datos

In [3]:
# Carguemos los datos:
data = pd.read_csv('players.csv')
data.head()

,Name,Age,Overall,Wage,Preferred Foot,International Reputation,Weak Foot,Skill Moves,LS,ST,...,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle
0,L. Messi,31,94,565000,Left,5.0,4.0,4.0,90.0,90.0,...,94.0,48.0,22.0,94.0,94.0,75.0,96.0,33.0,28.0,26.0
1,Cristiano Ronaldo,33,94,405000,Right,5.0,4.0,5.0,94.0,94.0,...,93.0,63.0,29.0,95.0,82.0,85.0,95.0,28.0,31.0,23.0
2,Neymar Jr,26,92,290000,Right,5.0,5.0,5.0,87.0,87.0,...,82.0,56.0,36.0,89.0,87.0,81.0,94.0,27.0,24.0,33.0
3,K. De Bruyne,27,91,355000,Right,4.0,5.0,4.0,85.0,85.0,...,91.0,76.0,61.0,87.0,94.0,79.0,88.0,68.0,58.0,51.0
4,E. Hazard,27,91,340000,Right,4.0,4.0,4.0,86.0,86.0,...,80.0,54.0,41.0,87.0,89.0,86.0,91.0,34.0,27.0,22.0


In [ ]:
# Veamos la descripción del dataset:
### CÓDIGO AQUÍ ###


In [ ]:
#Veamos la distribución de las columnas
### CÓDIGO AQUÍ ###

In [ ]:
# Obtengamos la distribución de valores de la variable `Preferred Foot`:
# tip: pandas function value_counts()
### CÓDIGO AQUÍ ###


Right    11969
Left      3957
Name: Preferred Foot, dtype: int64

## Regresión

**Nota:** Dejaremos la coulmna `Name` en el datafrme para poder identificar a los jugadores, pero **la debemos excluir** al definir las variables `X_train`, `y_train`, `X_val`, `y_val`, `X_test`, and `y_test`.

In [ ]:
# Separemos los datos en conjuntos de entrenamiento (80 %) y prueba (20 %):
### CÓDIGO AQUÍ ###

In [ ]:
#Definimos las transformaciones
cat_cols = X_train.select_dtypes(include=object).columns
num_cols = X_train.select_dtypes(include=np.number).columns

categorical_transformer = ### CÓDICO AQUÍ### 
numerical_transformer = 'passthrough'

preprocessor = ### CÓDIGO AQUÍ ###

#Definimos nuestro regresor
rf_base = RandomForestRegressor(random_state=random_state)
gb_base = GradientBoostingRegressor(random_state=random_state)

#Definimos los pipelines
pipeline_rf = ### CÓDIGO AQUÍ ###
pipeline_gb = ### CÓDIGO AQUÍ ### 

# Definamos la malla de parámetros sobre la que haremos la búsqueda:
param_grid = ### CÓDIGO AQUÍ ###

# Definamos nuestros modelo mediante GridSearchCV:
rf = GridSearchCV(pipeline_rf, cv=3, param_grid=param_grid)
gb = GridSearchCV(pipeline_gb, cv=3, param_grid=param_grid)

In [ ]:
%%time
# Entrenemos el random forest:
rf.fit(X_train, y_train)
# Entrenemos el gradient boosting:
gb.fit(X_train, y_train)

In [ ]:
print(rf.best_params_)
print(gb.best_params_)

In [ ]:
# Obtengamos el R^2 y el MAE de entrenamiento para cada modelo:
print("Train set")
for model in (rf, gb):
    print(f"Model: {'Random Forest' if model == rf else 'Gradient Boosting'}")
    print(f'R^2: {model.score(X_train, y_train)}')
    print(f'MAE: {mean_absolute_error(y_train, model.predict(X_train))}')
    print('\n')

print("Test set")
for model in (rf, gb):
    print(f"Model: {'Random Forest' if model == rf else 'Gradient Boosting'}")
    print(f'R^2: {model.score(X_test, y_test)}')
    print(f'MAE: {mean_absolute_error(y_test, model.predict(X_test))}')
    print('\n')

In [ ]:
# Veamos los datos junto con las predicciones:
pd.concat([
    data.drop(columns='Wage').reset_index(drop=True),
    data['Wage'].reset_index(drop=True),
    pd.DataFrame({'rf_predicted_wage': rf.predict(data.drop(columns=['Name', 'Wage']))}),
    pd.DataFrame({'gb_predicted_wage': gb.predict(data.drop(columns=['Name', 'Wage']))})
], axis=1)

Explicabilidad

In [ ]:
!pip install shap

In [ ]:
import shap
from IPython.display import HTML

In [ ]:
# Entrenamos nuestro pipeline con los mejores parámetros encontrados en la validación cruzada
pipeline_gb.set_params(**gb.best_params_)
pipeline_gb.fit(X_train, y_train)

In [ ]:
# Aplicamos el preproceso a los datos
X_train_preprocessed = ### CÓDIGO AQUÍ ###
X_test_preprocessed = ### CÓDIGO AQUÍ ###

In [ ]:
# Entrenemos nuestro modelo de explicabilidad
gb_explainer = ### CÓDIGO AQUÍ ###
train_gb_shap_values = ### CÓDIGO AQUÍ ###

In [ ]:
# Summary plot
### CÓDIGO AQUÍ ###

In [ ]:
# Grafiquemos los valores SHAP para un ejemplo del conjunto de test
### CÓDIGO AQUÍ ###

In [ ]:
# Grafiquemos la dependencia de los valores SHAP con la variable "Age"